In [1]:
import jax
from evox import Problem, algorithms,jit_class, monitors, workflows
from evox.operators import mutation, crossover, selection
from jax import random
import jax.numpy as jnp

In [2]:
Pd=10500
    
a=jnp.array([0.0069, 0.0069, 0.02028, 0.00942, 0.0114, 0.01142, 0.00357, 0.00492,
                        0.00573, 0.00605, 0.00515, 0.00569, 0.00421, 0.00752, 0.00708, 0.00708,
                        0.0031, 0.00313, 0.00313, 0.00313, 0.00298, 0.00298, 0.00284, 0.00284, 0.00277,
                        0.00277, 0.52124, 0.52124, 0.52124, 0.0114, 0.0016, 0.0016, 0.0016, 0.0001,
                        0.0001, 0.0001, 0.0161, 0.0161, 0.0161, 0.00313])
b=jnp.array([6.7300, 6.7300, 7.0700, 8.1800, 5.3500, 8.0500, 8.0300, 6.9900, 6.6000, 12.9000,
                        12.9000, 12.8000, 12.5000, 8.8400, 9.1500, 9.1500, 7.9700, 7.9500, 7.9700, 7.9700,
                         6.6300, 6.6300, 6.6600, 6.6600, 7.1000, 7.1000, 3.3300, 3.3300, 3.3300, 5.3500,
                         6.4300, 6.4300, 6.4300, 8.9500, 8.6200, 8.6200, 5.8800, 5.8800, 5.8800, 7.9700])
c=jnp.array([0.094705, 0.094705, 0.30954, 0.36903, 0.14889, 0.22233, 0.28771, 0.39198,
                        0.45576, 0.72282, 0.6352, 0.65469, 0.9134, 1.7604, 1.7283, 1.7283, 0.64785,
                        0.64969, 0.64783, 0.64781, 0.78596, 0.78596, 0.79453, 0.79453, 0.80132, 0.80132,
                        1.0551, 1.0551, 1.0551, 0.14889, 0.22292, 0.22292, 0.22292, 0.10787, 0.11658,
                        0.11658, 0.30745, 0.30745, 0.30745, 0.64783]) * 1000
g=jnp.array([
                100, 100, 100, 150, 120, 100, 200, 200, 200, 200, 200, 200, 300, 300, 300, 300,
                300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 120, 120, 120, 120, 150, 150,
                150, 200, 200, 200, 80, 80, 80, 300
            ])

h=jnp.array([
                0.0840, 0.0840, 0.0840, 0.0630, 0.0770, 0.0840, 0.0420, 0.0420, 0.0420, 0.0420,
                0.0420, 0.0420, 0.0350, 0.0350, 0.0350, 0.0350, 0.0350, 0.0350, 0.0350, 0.0350,
                0.0350, 0.0350, 0.0350, 0.0350, 0.0350, 0.0350, 0.0770, 0.0770, 0.0770, 0.0770,
                0.0630, 0.0630, 0.0630, 0.0420, 0.0420, 0.0420, 0.0980, 0.0980, 0.0980, 0.0350
            ])
lb=jnp.array([36, 36, 60, 80, 47, 68, 110, 135, 135, 130, 94, 94, 125, 125, 125,
                           125, 220, 220, 242, 242, 254, 254, 254, 254, 254, 254, 10, 10, 10,
                           47, 60, 60, 60, 90, 90, 90, 25, 25, 25, 242])
ub=jnp.array([114, 114, 120, 190, 97, 140, 300, 300, 300, 300, 375, 375, 500, 500,
                           500, 500, 500, 500, 550, 550, 550, 550, 550, 550, 550, 550, 150, 150,
                           150, 97, 190, 190, 190, 200, 200, 200, 110, 110, 110, 550])

In [3]:
def initialize_center_with_constraints(key, Pd, lb, ub):
    n = lb.shape[0]  # 获取向量的长度
    # 1. 随机生成一个在 [lb, ub] 之间的初始向量
    random_init = jax.random.uniform(key, shape=(n,), minval=0.0, maxval=1.0)
    
    # 2. 通过线性插值将 random_init 映射到 [lb, ub] 之间
    scaled_init = lb + random_init * (ub - lb)
    
    # 3. 计算当前随机向量的和
    sum_scaled_init = jnp.sum(scaled_init)
    
    # 4. 将向量调整为和等于 Pd
    center_init = scaled_init * (Pd / sum_scaled_init)  # 重新缩放以使其和为 Pd
    
    # 5. 确保调整后的值仍然在 [lb, ub] 之间
    center_init = jnp.clip(center_init, lb, ub)
    
    return center_init

key = random.PRNGKey(2)
center_init = initialize_center_with_constraints(key, Pd, lb, ub)
print(center_init)

[ 50.452946  71.743164 120.       190.        97.        94.217514
 165.93658  300.       300.       197.93858  375.       120.08821
 365.22363  181.1079   201.60548  500.       500.       500.
 550.       550.       405.20096  517.7552   550.       346.80945
 519.1493   401.31702   87.787254 116.93457  150.        80.10183
 143.02684  138.74217  190.       118.06074  112.88136  185.89024
 110.        65.95288   50.843006 299.49225 ]


In [4]:
print(sum(center_init))

10020.26


In [5]:


def adjust_population_average(x, Pd, lb, ub):#单次平均调整
    n = x.shape[0]  # 获取发电机的数量  
    sumpop = jnp.sum(x)  # 计算所有发电机功率的总和
    df = sumpop - Pd  # 计算功率偏差

    # 计算每个发电机需要减少的平均功率
    df_per_generator = df / n

    new_df_per_generator = jnp.clip(df_per_generator,lb,ub)  # 计算新的每个发电机需要减少的平均功率，确保在范围内
    
    # 从每个发电机的功率中减去平均偏差
    x_adjusted = x - new_df_per_generator

    # 确保更新后的值逐元素不超出上下限
    x_adjusted = jnp.clip(x_adjusted, lb, ub)
    
    return x_adjusted


def adjust_population_average_loop(x, Pd, key, lb, ub, tolerance=1e-1): #多次平均调整
    n = x.shape[0]  # 获取发电机的数量

    def cond_fun(state):
        x, sumpop, df, r = state
        return jnp.abs(sumpop - Pd) >= tolerance
    
    def body_fun(state):
        x, sumpop, df, r = state
        df = sumpop - Pd  # 计算功率偏差        
        df_per_generator = df / n  # 计算每个发电机需要减少的平均功率

        new_df_per_generator = jnp.clip(df_per_generator,lb,ub)

        x =  x - df_per_generator  # 从每个发电机的功率中减去平均偏差
        x = jnp.clip(x, lb, ub)  # 确保更新后的值不超出上下限

        sumpop = jnp.sum(x)  # 计算更新后所有发电机功率的总和
        return x, sumpop, df, r

    state = (x, jnp.sum(x), 0, 0)
    x, _, _, _ = jax.lax.while_loop(cond_fun, body_fun, state)
    return x



def adjust_population_random(x, Pd, key, lb, ub):   #单次随机调整
    n = x.shape[0]  # 获取发电机的数量  
    sumpop = jnp.sum(x)  # 计算所有发电机功率的总和
    df = sumpop - Pd  # 计算功率偏差

    r = random.randint(key, shape=(), minval=0, maxval=n+1) #生成随机数r,选择对第r个发电机进行调整

    new_value = x[r] - df  # 计算新的功率值
    new_value = jnp.clip(new_value, lb[r], ub[r])  # 确保新值在 lb[r] 和 ub[r] 之间
    # 从第r个发电机中减去总偏差
    x = x.at[r].set(new_value)  # 使用 .at[].set() 更新特定索引的值

    # 确保更新后的值不超出上下限
    x = jnp.clip(x, lb, ub)

    return x


def adjust_population_random_loop(x, Pd, key, lb, ub, tolerance):  # 多次随机调整
    n = x.shape[0]  # 获取发电机的数量

    def cond_fun(state):
        x, sumpop, df, r = state
        return jnp.abs(sumpop - Pd) >= tolerance

    def body_fun(state):
        x, sumpop, df, r = state
        df = sumpop - Pd  # 计算功率偏差
        r = random.randint(key, shape=(), minval=0, maxval=n+1)  #生成随机数r,选择对第r个发电机进行调整

        new_value = x[r] - df  # 计算新的功率值
        new_value = jnp.clip(new_value, lb[r], ub[r])  # 确保新值在 lb[r] 和 ub[r] 之间
        # 从第r个发电机中减去总偏差
        x = x.at[r].set(new_value)  # 使用 .at[].set() 更新特定索引的值
        # x = jnp.clip(x, lb, ub)  # 确保更新后的值不超出上下限
        sumpop = jnp.sum(x)  # 计算更新后所有发电机功率的总和
        return x, sumpop, df, r

    state = (x, jnp.sum(x), 0, 0)
    x, _, _, _ = jax.lax.while_loop(cond_fun, body_fun, state)
    return x


def _eld_openes(x):

    #x = adjust_population_average(x, Pd, lb, ub) #单次平均调整
    key = random.PRNGKey(42)
    #x = adjust_population_random(x, Pd, key, lb, ub) #随机选择发电机调整
    #x = adjust_population_average(x, Pd, lb, ub) #机组平均调整

    x = adjust_population_random_loop(x, Pd, key, lb, ub, tolerance=10)#!!!多次随机选择发电机调整

    #x = adjust_population_average_loop(x, Pd, key, lb, ub, tolerance=1e-1) #多次平均调整发电机组功率

    #x = jnp.clip(x, lb, ub)  # 确保更新后的值不超出上下限
            
    return ((a * (x ** 2) + b * x + c).sum() + jnp.abs(g * jnp.sin(h * (Pd - x))).sum())



def eld_openes(x):
    return jax.vmap(_eld_openes, in_axes= 0)(x)



# @evox.jit_class
class ELD_OpenES(Problem):
    def __init__(self):
        super().__init__()
 
    def evaluate(self, State, x):
        return eld_openes(x), State

In [6]:
algorithm = algorithms.OpenES(
    center_init = center_init,
    pop_size=5000,
    learning_rate=0.1,
    noise_stdev=0.001,
    optimizer="adam",
    mirrored_sampling=False,
    )

In [7]:
#创建一个problem实例
Problem = ELD_OpenES()

monitor = monitors.EvalMonitor()
# create a workflow
workflow = workflows.StdWorkflow(
    algorithm,
    Problem,
    monitors=[monitor],
)

In [22]:
import time

# init the workflow
key = jax.random.PRNGKey(4)
state = workflow.init(key)

start_time = time.time()
# run the workflow for 100 steps
for i in range(500):
    state = workflow.step(state)

end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

In [9]:
monitor.get_best_fitness()

Array(78547.836, dtype=float32)

In [10]:
monitor.get_best_solution()

Array([  3.9856544,  15.13059  ,  53.299667 , 118.076065 ,  33.815697 ,
        31.581768 ,  95.27907  , 224.99397  , 224.99368  , 125.668915 ,
       298.96808  ,  53.376766 , 289.30142  , 109.60811  , 129.18546  ,
       422.96606  , 422.96802  , 422.96796  , 472.69974  , 472.70004  ,
       328.871    , 440.62375  , 472.69852  , 271.109    , 442.0089   ,
       325.02203  ,  26.556095 ,  50.61773  ,  80.41655  ,  20.853867 ,
        73.99912  ,  70.08559  , 118.07516  ,  51.60221  ,  47.10412  ,
       114.155464 ,  44.636738 ,  11.535713 ,   4.1298137, 224.49368  ],      dtype=float32)

In [11]:
value = monitor.get_best_solution()

lb= jnp.array([36, 36, 60, 80, 47, 68, 110, 135, 135, 130, 94, 94, 125, 125, 125,
                           125, 220, 220, 242, 242, 254, 254, 254, 254, 254, 254, 10, 10, 10,
                           47, 60, 60, 60, 90, 90, 90, 25, 25, 25, 242])
ub= jnp.array([114, 114, 120, 190, 97, 140, 300, 300, 300, 300, 375, 375, 500, 500,
                           500, 500, 500, 500, 550, 550, 550, 550, 550, 550, 550, 550, 150, 150,
                           150, 97, 190, 190, 190, 200, 200, 200, 110, 110, 110, 550])
for i, value in enumerate(value):
    if value < lb[i] or value > ub[i]:
        print(f"Value {value} at index {i} is out of bounds.")

Value 3.985654354095459 at index 0 is out of bounds.
Value 15.130590438842773 at index 1 is out of bounds.
Value 53.29966735839844 at index 2 is out of bounds.
Value 33.815696716308594 at index 4 is out of bounds.
Value 31.581768035888672 at index 5 is out of bounds.
Value 95.27906799316406 at index 6 is out of bounds.
Value 125.66891479492188 at index 9 is out of bounds.
Value 53.376766204833984 at index 11 is out of bounds.
Value 109.60810852050781 at index 13 is out of bounds.
Value 20.853866577148438 at index 29 is out of bounds.
Value 51.602210998535156 at index 33 is out of bounds.
Value 47.10411834716797 at index 34 is out of bounds.
Value 11.535713195800781 at index 37 is out of bounds.
Value 4.1298136711120605 at index 38 is out of bounds.
Value 224.49368286132812 at index 39 is out of bounds.


In [12]:
# value = monitor.get_best_solution()
value = monitor.get_best_solution()
total_sum = sum(value)

# 检查总和是否等于目标值
if total_sum == 10500:
    print("总和等于目标值。")
else:
    print(f"总和不等于目标值，总和为：{total_sum}，目标值为：{10500}")

总和不等于目标值，总和为：7240.16064453125，目标值为：10500


In [13]:
# dif = total_sum - 10500
# print(f"总和不等于目标值，总和为：{total_sum}，目标值为：{10500}，差值为：{dif}")

# dif = dif / 40.0
# print(f"每台发电机组功率调整：{dif}")

# x_final = monitor.get_best_solution() - dif

# sum_final = sum(x_final)
# print(sum_final)
# cost_final =  ((a * (x_final ** 2) + b * x_final + c).sum() + jnp.abs(g * jnp.sin(h * (10500 - x_final))).sum())
# print(f"最终成本：{cost_final}")

In [14]:
# 1. 获取最优解
x_final = monitor.get_best_solution()

# 2. 开始调整循环，直到满足总和为10500且元素在上下界
tolerance = 1e-3  # 设置一个容差，确保浮点运算不会导致无限循环
target_sum = 10500

while True:
    # 计算当前总和的偏差
    total_sum = jnp.sum(x_final)
    dif = total_sum - target_sum
    print(f"总和不等于目标值，总和为：{total_sum}，目标值为：{target_sum}，差值为：{dif}")

    # 计算每个发电机需要调整的功率
    dif_per_generator = dif / 40.0
    print(f"每台发电机组功率调整：{dif_per_generator}")

    # 3. 减去功率偏差
    x_final = x_final - dif_per_generator

    # 4. 将 x_final 限制在 lb 和 ub 之间
    x_final = jnp.clip(x_final, lb, ub)

    # 5. 重新计算缩放后的总和
    sum_final = jnp.sum(x_final)

    # 如果缩放后的总和仍然不等于 10500，重新进行缩放调整
    if jnp.abs(sum_final - target_sum) < tolerance:
        print(f"调整后的总和在容差范围内：{sum_final}")
        break  # 如果总和已经在容差范围内，则退出循环

    # 如果总和不等于10500，则通过缩放调整整个数组
    scaling_factor = target_sum / sum_final
    x_final = x_final * scaling_factor

    # 确保缩放后的 x_final 仍然在 lb 和 ub 之间
    x_final = jnp.clip(x_final, lb, ub)

# 6. 重新计算总和和成本
sum_final = jnp.sum(x_final)
print(f"最终的总和: {sum_final}")

# 计算最终成本
cost_final = ((a * (x_final ** 2) + b * x_final + c).sum() + 
              jnp.abs(g * jnp.sin(h * (target_sum - x_final))).sum())
print(f"最终成本：{cost_final}")

总和不等于目标值，总和为：7240.162109375，目标值为：10500，差值为：-3259.837890625
每台发电机组功率调整：-81.4959487915039
总和不等于目标值，总和为：10436.296875，目标值为：10500，差值为：-63.703125
每台发电机组功率调整：-1.5925781726837158
总和不等于目标值，总和为：10487.4609375，目标值为：10500，差值为：-12.5390625
每台发电机组功率调整：-0.3134765625
总和不等于目标值，总和为：10497.3193359375，目标值为：10500，差值为：-2.6806640625
每台发电机组功率调整：-0.0670166015625
总和不等于目标值，总和为：10499.427734375，目标值为：10500，差值为：-0.572265625
每台发电机组功率调整：-0.01430664025247097
总和不等于目标值，总和为：10499.876953125，目标值为：10500，差值为：-0.123046875
每台发电机组功率调整：-0.0030761719681322575
总和不等于目标值，总和为：10499.974609375，目标值为：10500，差值为：-0.025390625
每台发电机组功率调整：-0.0006347656017169356
总和不等于目标值，总和为：10499.994140625，目标值为：10500，差值为：-0.005859375
每台发电机组功率调整：-0.0001464843808207661
总和不等于目标值，总和为：10499.998046875，目标值为：10500，差值为：-0.001953125
每台发电机组功率调整：-4.882812572759576e-05
调整后的总和在容差范围内：10500.0
最终的总和: 10500.0
最终成本：154098.328125
